In [1]:
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

import numpy as np
import pandas as pd
import deepchem as dc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_max_pool
from torch_geometric.data import Batch, Data
from torch_geometric.loader import DataLoader
from sklearn import metrics

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading some Jax models, missing a dependency. No module named 'haiku'


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(30, 256)
        self.conv2 = GCNConv(256, 256)
        self.conv3 = GCNConv(256, 256)
        self.conv4 = GCNConv(256, 256)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(p=0.55)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.leaky_relu(x)
        x = self.conv3(x, edge_index)
        x = F.leaky_relu(x)
        x = self.conv4(x, edge_index)
        x = F.leaky_relu(x)
        x = global_max_pool(x, data.batch)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        return x

In [3]:
def custom_collate(batch):
    data_list, target_list = zip(*batch)
    batch_data = Batch.from_data_list(data_list)
    batch_target = torch.stack(target_list)
    return batch_data, batch_target

In [4]:
def calculate_statistics(group):
    r2_train = group['r2_train']
    r2_test = group['r2_test']
    return pd.Series({
        'r2_train_mean': np.mean(r2_train),
        'r2_train_std': np.std(r2_train, ddof=0),
        'r2_test_mean': np.mean(r2_test),
        'r2_test_std': np.std(r2_test, ddof=0)
    })

In [5]:
torch.manual_seed(0)

scaler = StandardScaler()

epochs = 100
lr = 1e-2
wd = 3e-4

results = []
for random_state in range(10):
    torch.manual_seed(0)
    print(f'Random State: {random_state}')
    
    for d in ["AI", "AI2", "Random", "Human"]:
        torch.manual_seed(0)
        print('Source : ', d)
        dataset = d
        
        for c in ["Yield_CO"]:
            torch.manual_seed(0)
            df = pd.read_csv('data_Real/data_real.csv')
            smiles = df["SMILES"]
            featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
            X = featurizer.featurize(smiles)
            
            y = df[c]
            data_train, data_test, target_train, target_test = train_test_split(X, y, test_size=0.5, random_state=random_state)

            target_train = scaler.fit_transform(target_train.values.reshape(-1, 1)).flatten()
            target_test = scaler.transform(target_test.values.reshape(-1, 1)).flatten()
            
            target_train = torch.tensor(target_train, dtype=torch.float32)
            target_test = torch.tensor(target_test, dtype=torch.float32)
            
            data_train_list = []
            for graph_data in data_train:
                node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
                edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
                edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
                data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
                data_train_list.append(data)

            data_test_list = []
            for graph_data in data_test:
                node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
                edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
                edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
                data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
                data_test_list.append(data)

            train_loader = DataLoader(list(zip(data_train_list, target_train)), batch_size=len(data_train_list), collate_fn=custom_collate)
            test_loader = DataLoader(list(zip(data_test_list, target_test)), batch_size=len(data_test_list), collate_fn=custom_collate)

            model = Net()
            model.load_state_dict(torch.load(f'data_{dataset}/model_{dataset}_sc.pth'))
            model.fc3 = nn.Linear(128, 1)
        
            model.train()
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            criterion = nn.MSELoss()
            
            for param in model.conv1.parameters():
                param.requires_grad = False
            for param in model.conv2.parameters():
                param.requires_grad = False
            for param in model.conv3.parameters():
                param.requires_grad = False
            for param in model.conv4.parameters():
                param.requires_grad = False

            device = torch.device('cpu')
            model.to(device)
            
            # Replacing the head layer with a new one.
            # Unfreezing FC layers.

            for epoch in range(epochs):
                for data, target in train_loader:
                    data = data.to(device)
                    target = target.to(device)
                    optimizer.zero_grad()
                    out = model(data)
                    loss = criterion(out, target.view(-1, 1))
                    loss.backward()
                    optimizer.step()

            model.eval()
            pred_train = []
            for data, target in train_loader:
                data = data.to(device)
                with torch.no_grad():
                    out = model(data)
                pred_train.append(out.cpu().numpy())
            pred_train = np.concatenate(pred_train)

            pred_test = []
            for data, target in test_loader:
                data = data.to(device)
                with torch.no_grad():
                    out = model(data)
                pred_test.append(out.cpu().numpy())
            pred_test = np.concatenate(pred_test)

            pred_train = scaler.inverse_transform(pred_train)
            pred_test = scaler.inverse_transform(pred_test)
            target_train = scaler.inverse_transform(target_train.numpy().reshape(-1, 1)).flatten()
            target_test = scaler.inverse_transform(target_test.numpy().reshape(-1, 1)).flatten()

            r2_train_score = metrics.r2_score(target_train, pred_train)
            r2_test_score = metrics.r2_score(target_test, pred_test)

            results.append({'source': d, 'target': c, 'r2_train': r2_train_score, 'r2_test': r2_test_score})
            print(f'R2 test for {c} with random state {random_state}: {r2_test_score}')

results_df = pd.DataFrame(results)
gen_results = results_df.groupby(['source', 'target']).apply(calculate_statistics).reset_index()
gen_results

Random State: 0
Source :  AI
R2 test for Yield_CO with random state 0: 0.7569352984428406
Source :  AI2
R2 test for Yield_CO with random state 0: 0.7016013860702515
Source :  Random
R2 test for Yield_CO with random state 0: 0.8318687081336975
Source :  Human
R2 test for Yield_CO with random state 0: 0.6816807389259338
Random State: 1
Source :  AI
R2 test for Yield_CO with random state 1: 0.5810601711273193
Source :  AI2
R2 test for Yield_CO with random state 1: 0.6035688519477844
Source :  Random
R2 test for Yield_CO with random state 1: 0.7316558361053467
Source :  Human
R2 test for Yield_CO with random state 1: 0.7454807758331299
Random State: 2
Source :  AI
R2 test for Yield_CO with random state 2: 0.49990028142929077
Source :  AI2
R2 test for Yield_CO with random state 2: 0.324124813079834
Source :  Random
R2 test for Yield_CO with random state 2: 0.737076997756958
Source :  Human
R2 test for Yield_CO with random state 2: 0.6842539310455322
Random State: 3
Source :  AI
R2 test for 

,source,target,r2_train_mean,r2_train_std,r2_test_mean,r2_test_std
0,AI,Yield_CO,0.976645,0.010410,0.709990,0.110004
1,AI2,Yield_CO,0.974372,0.011643,0.676897,0.158006
2,Human,Yield_CO,0.967810,0.022188,0.743448,0.051074
3,Random,Yield_CO,0.979342,0.007806,0.800274,0.058002


In [6]:
gen_results.to_csv(f'result/result_yield_leakyReLU.csv')